In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import imutils
import cv2
import os

In [2]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
face = "D:/ZaloAI/LIVENESS/model/face_detection"
prototxtPath =  os.path.sep.join([face, "deploy.prototxt"])
weightsPath = os.path.sep.join([face,
	"res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

[INFO] loading face detector model...


In [8]:
# def detect_and_predict_mask(frame, faceNet, maskNet):
def get_face(frame, faceNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			if face.any():
				# face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
				face = cv2.resize(face, (32, 32))

				# add the face and bounding boxes to their respective
				# lists
				faces.append(face)

	return faces

In [4]:
def save_image(image,addr,num):
    address = addr + str(num)+ '.jpg'
    cv2.imwrite(address,image)

In [11]:
source = "D:/ZaloAI/LIVENESS/dataset/fake_vids"
destination = "D:/ZaloAI/LIVENESS/dataset/fake_img"

In [12]:
for filename in os.scandir(source):
    i, f_count = (0, 0)
    name = filename.name.split(".")[0]
    cap = cv2.VideoCapture(filename.path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # loop over the frames from the video stream
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        f_count += 1
        if f_count % fps != 0:
            continue
        
        # grab the frame from the threaded video stream and resize it
        # to have a maximum width of 400 pixels
        i += 1
        frame = imutils.resize(frame, width=400)

        # detect faces in the frame and determine if they are wearing a
        # face mask or not
        faces = get_face(frame, faceNet)
        for face in faces:
            save_image(face, (destination + '/' + name + '_'), i)
        
        # key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        # if key == ord("q"):
            # break

    # do a bit of cleanup
    cap.release()
    cv2.destroyAllWindows()